In [1]:
import sys, os, time
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
%matplotlib notebook
# %matplotlib tk
import matplotlib.pyplot as plt
import imageio
import seaborn as sns
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data

np.set_printoptions(linewidth=120, suppress=True)

In [2]:
M = train.Run_Manager(tbout='/is/ei/fleeb/workspace/trained_nets/managed')
M.clear_links()
M.start_tb()

/is/ei/fleeb/workspace/trained_nets/managed is available to view runs on tensorboard


/is/ei/fleeb/workspace/foundation/foundation/train/config.py:58: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


Found 137 runs
Tensorboard started: http://localhost:6006/


In [3]:
#M.show()
M.clear_filters().select(model=None, dataset='3dshapes').filter(lambda r: not 'test' in r.name)
# M.filter(lambda r: '191224' in r.name)
# M.filter(lambda r: '191218' in r.name)

M.filter_idx(7, 5, 27, 23, 34, 16, 15, 13, 30, 28, 10, 12, 18, 21, 0, 1, 72, 73, 74, 67, 66, 57)


M.show()

  0 - 3dshapes-dvae_0077-5920872-45_191213-201528
  1 - 3dshapes-dvae_0087-5930025-01_191219-182417
  2 - 3dshapes-dwae_0077-5920872-43_191213-201526
  3 - 3dshapes-dwae_0087-5930025-00_191219-182415
  4 - 3dshapes-fdvae_0077-5920872-50_191213-201526
  5 - 3dshapes-fdvae_0082-5923472-13_191218-100112
  6 - 3dshapes-fdwae_0077-5920872-46_191213-201528
  7 - 3dshapes-fdwae_0077-5920872-48_191213-201525
  8 - 3dshapes-fdwae_0082-5923472-12_191218-100112
  9 - 3dshapes-fvae_0077-5920872-37_191213-201524
 10 - 3dshapes-fvae_0077-5920872-40_191213-201528
 11 - 3dshapes-fwae_0077-5920872-33_191213-201527
 12 - 3dshapes-fwae_0088-5943512-01_191224-232233
 13 - 3dshapes-vae_0077-5920872-21_191213-201539
 14 - 3dshapes-vae_0077-5920872-23_191213-201527
 15 - 3dshapes-wae_0077-5920872-10_191213-201528
 16 - 3dshapes-wgan_0080-5920891-2_191214-090739
 17 - 3dshapes-wgan_0085-5925596-08_191218-171727
 18 - 3dshapes-wgan_0085-5925596-09_191218-171726
 19 - 3dshapes-wpp_0077-5920872-24_191213-201524


In [4]:
ignore = None
# ignore = ['disc_steps']
M.show_unique(ignore)

0) 3dshapes-dvae_0077-5920872-45_191213-201528
	model.disc_steps - 1 (5)

1) 3dshapes-dvae_0087-5930025-01_191219-182417
	model.prob - 0.5 (1)

2) 3dshapes-dwae_0077-5920872-43_191213-201526
	model.disc_steps - 1 (5)

3) 3dshapes-dwae_0087-5930025-00_191219-182415
	model.prob - 0.5 (1)

4) 3dshapes-fdvae_0077-5920872-50_191213-201526
	model.disc_steps - 1 (5)
	model.gan_wt - 0.001 (0.0001)

5) 3dshapes-fdvae_0082-5923472-13_191218-100112
	model.disc_steps - 1 (5)
	model.prob - 0.5 (1)

6) 3dshapes-fdwae_0077-5920872-46_191213-201528
	model.disc_steps - 1 (5)

7) 3dshapes-fdwae_0077-5920872-48_191213-201525
	model.disc_steps - 1 (5)
	model.gan_warm_start - 20 (_)

8) 3dshapes-fdwae_0082-5923472-12_191218-100112
	model.disc_steps - 1 (5)
	model.prob - 0.5 (1)

9) 3dshapes-fvae_0077-5920872-37_191213-201524
	model.disc_steps - 1 (5)

10) 3dshapes-fvae_0077-5920872-40_191213-201528
	model.disc_steps - 1 (5)
	model.gan_wt - 0.0001 (0)

11) 3dshapes-fwae_0077-5920872-33_191213-201527
	model.

In [5]:
M.link('{idx}__{model}__{unique}')

In [6]:
# M.link('{name}__{unique}')

In [6]:
# os.listdir(M.tbout)

In [9]:
M.clear_links()

In [8]:
M.tbout

'/is/ei/fleeb/workspace/trained_nets/managed'

In [18]:
import json

In [19]:
json.loads('[10,2,3]')

[10, 2, 3]

In [ ]:
saveroot = os.environ['FOUNDATION_SAVE_DIR']
saveroot

In [ ]:
for name in os.listdir(saveroot):
    path = os.path.join(saveroot, name)
    if 'config.yml' in os.listdir(path):
        print(name,end=' ')
        with open(os.path.join(path, 'config.yml'), 'r') as f:
            data = yaml.load(f)
        break
#         if 'parents' in data:
#             del data['parents']
#             with open(os.path.join(path, 'config.yml'), 'w') as f:
#                 yaml.dump(data, f)
#             print('fixed')
#         else:
#             print()

In [ ]:
data['parents']